In [ ]:
# SETUP COMPLETO
import os
os.chdir('/content')
!rm -rf Botardo
!git clone https://github.com/TobiasActis/Botardo.git
os.chdir('/content/Botardo')
print("✅", os.getcwd())
!pip install -q pandas numpy matplotlib loguru ta requests python-binance

In [ ]:
# DESCARGAR DATOS
!python download_btc_futures_1m.py

In [ ]:
# BACKTEST 2 AÑOS
!python backtest_superbot.py --data_1m "data/BTCUSDT_1m_2024-01-01_to_now.csv" --initial_capital 500 --risk_per_trade 0.10 --leverage 50 --po3_min_rr 3.5 --smc_standalone 8 --start "2024-01-01" --end "2025-12-09"

In [ ]:
# GRÁFICO
from IPython.display import Image, display
import os
if os.path.exists('superbot_backtest.png'):
    display(Image('superbot_backtest.png', width=900))

In [ ]:
# ANÁLISIS
import pandas as pd
import os

if not os.path.exists('superbot_backtest_trades.csv'):
    print("❌ No hubo trades")
else:
    t = pd.read_csv('superbot_backtest_trades.csv')
    w = t[t['pnl'] > 0]
    l = t[t['pnl'] < 0]
    
    print(f"Trades: {len(t)}")
    print(f"Win Rate: {len(w)/len(t)*100:.1f}%")
    print(f"P&L: ${t['pnl'].sum():.2f}")
    print(f"ROI: {t['pnl'].sum()/500*100:.1f}%")
    print(f"Best: ${t['pnl'].max():.2f}")
    print(f"Worst: ${t['pnl'].min():.2f}")
    
    # BUG CHECK
    lb = t[(t['side']=='LONG') & (t['stop_loss']>t['entry_price'])]
    sb = t[(t['side']=='SHORT') & (t['stop_loss']<t['entry_price'])]
    if len(lb)+len(sb) > 0:
        print(f"\n❌ BUG: {len(lb)+len(sb)} trades con SL invertido")
    else:
        print("\n✅ SL correcto en todos los trades")
    
    print("\nÚltimos 10:")
    print(t[['timestamp','side','entry_price','exit_price','pnl','pnl_pct','exit_reason']].tail(10))

In [ ]:
# DESCARGAR
from google.colab import files
import os
if os.path.exists('superbot_backtest.png'):
    files.download('superbot_backtest.png')
if os.path.exists('superbot_backtest_trades.csv'):
    files.download('superbot_backtest_trades.csv')
print("✅ Descargado")